<a href="https://www.kaggle.com/code/adinathjagtap777/titanic-machine-learning-from-disaster?scriptVersionId=290545124" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Cell 1: Setup environment
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#imports
import numpy as np 
import pandas as pd

In [2]:
# Cell 2: Import libraries
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import tensorflow_decision_forests as tfdf

E0000 00:00:1767793716.894402      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767793717.002051      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767793717.827319      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767793717.827378      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767793717.827383      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767793717.827387      17 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
# Cell 3: Load data
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

In [4]:
# Cell 4: Preprocessing
def preprocess(df):
    df = df.copy()
    df["Name"] = df["Name"].apply(lambda x: " ".join([v.strip(",()[].\"'") for v in x.split(" ")]))
    df["Ticket_number"] = df["Ticket"].apply(lambda x: x.split(" ")[-1])
    df["Ticket_item"] = df["Ticket"].apply(lambda x: "NONE" if len(x.split(" ")) == 1 else "_".join(x.split(" ")[0:-1]))
    return df

preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

In [5]:
# Cell 5: Feature selection
input_features = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']

In [6]:
# Cell 6: Create datasets

def tokenize_names(features, labels=None):
    features["Name"] = tf.strings.split(features["Name"])
    return features, labels


import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label="Survived").map(tokenize_names)
    serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)


In [7]:
# Cell 7: Train ensemble
predictions = None
num_predictions = 0

for i in range(100):
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0,
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True,
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1


predictions /= num_predictions

W0000 00:00:1767793741.105832      17 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1767793741.106950      17 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1767793741.106999      17 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1767793745.079762      17 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1767793745.079810      17 kernel.cc:783] Collect training examples
I0000 00:00:1767793745.079826      17 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

In [8]:
# Cell 8: Create submission
submission = pd.DataFrame({
    "PassengerId": serving_df["PassengerId"],
    "Survived": (predictions >= 0.5).astype(int)
})

submission.to_csv("/kaggle/working/submission.csv", index=False)
print("Done")

Done
